In [1]:
!git clone https://github.com/sorki/python-mnist
!./python-mnist/bin/mnist_get_data.sh
!pip3 install emnist keras-tuner
from emnist import extract_training_samples

print("Imported the EMNIST libraries we need!")

fatal: destination path 'python-mnist' already exists and is not an empty directory.
data directory already present, exiting
Imported the EMNIST libraries we need!


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.utils import to_categorical
from emnist import extract_training_samples, extract_test_samples


from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

import kerastuner

In [3]:
from tensorflow.python.client import device_lib

# Check all available devices if GPU is available
print(device_lib.list_local_devices())
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4974920139949652951
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14674281152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6788237759949135502
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



In [4]:
train_X, train_y = extract_training_samples("digits");
test_X, test_y = extract_test_samples("digits");

# Normalise
train_X = train_X.astype('float32')
train_X /= 255
test_X = test_X.astype('float32')
test_X /= 255

#train_X, val_X, train_y, val_y = train_test_split(images, labels, test_size= 0.10, random_state=42)


In [5]:
# Reshape image for CNN
train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

num_classes = np.unique(train_y)
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

In [ ]:
from kerastuner.applications import HyperResNet
from kerastuner.tuners import Hyperband

hypermodel = HyperResNet(input_shape=(28, 28, 1), classes=10)

tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=20,
    directory='my_dir',
    project_name='HyperResNet')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(train_X, train_y,
             epochs=20,
             validation_split=0.2,
             callbacks=[stop_early])

Trial 12 Complete [00h 29m 30s]
val_accuracy: 0.9934375286102295

Best val_accuracy So Far: 0.9934375286102295
Total elapsed time: 05h 16m 32s

Search: Running Trial #13

Hyperparameter    |Value             |Best Value So Far 
version           |v1                |v2                
conv3_depth       |4                 |4                 
conv4_depth       |36                |23                
pooling           |avg               |avg               
optimizer         |sgd               |sgd               
learning_rate     |0.01              |0.1               
tuner/epochs      |7                 |7                 
tuner/initial_e...|3                 |3                 
tuner/bracket     |2                 |2                 
tuner/round       |1                 |1                 
tuner/trial_id    |5c18ab014cd7a1d...|320d45db2d83950...

Epoch 4/7
6000/6000 [==============================] - 618s 101ms/step - loss: 0.5605 - accuracy: 0.8903 - val_loss: 0.0669 - val_accuracy: 0.97